In [4]:
!pip install --upgrade pip setuptools wheel
!pip install --no-cache-dir catboost


  Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.8.0
    Uninstalling setuptools-75.8.0:
      Successfully uninstalled setuptools-75.8.0
     ---------------------------------------- 0.0/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/71.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/71.5 MB ? eta -:--:--
     --------------------------------------- 0.5/71.5 MB 186.5 kB

  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [135 lines of output]
    Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
    Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
    Using cached jupyterlab-3.6.8-py3-none-any.whl.metadata (12 kB)
    Using cached conan-1.66.0.tar.gz (789 kB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached ipython-9.1.0-py3-none-any.whl.metadata (4.4 kB)
    Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
    Using cached tornado-6.4.2-cp38-abi3-win_amd64.whl.metadata (2.6 kB)
    Using cached jupyter_core-5.7.2-py3-none-any.whl.metadata (3.4 kB)
    Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
    Using cached jupyter_server-2.15.0-py3-none-any.whl.metadata (8.4 kB)
    Using cached jupyter_ydoc-0.2.5-py3-none-any.whl.metadata

In [1]:
def reformat_arff_file(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Find the index where @data starts
    data_start_index = None
    for i, line in enumerate(lines):
        if line.strip().lower() == '@data':
            data_start_index = i
            break

    if data_start_index is None:
        raise ValueError("No @data section found in the ARFF file.")

 
    data_lines = lines[data_start_index + 1:]  # Everything after @data

    #  adding line breaks after every row of data
    formatted_data = []
    for line in data_lines:
        line = line.strip()  # Remove extra spaces
        if line:  # Make sure it's not an empty line
            formatted_data.append(line)

    # Write into a new ARFF file
    with open(output_file, 'w') as f:
        # Write the header (everything before @data)
        for line in lines[:data_start_index + 1]:
            f.write(line)

        # Write the formatted data
        for row in formatted_data:
            f.write(row + '\n')

    print(f"Formatted ARFF file saved to: {output_file}")

# Example usage
input_file = r"E:\chronic_kidney_disease_full.arff"
output_file = r"E:\chronic_kidney_disease_full_cleaned.arff"

reformat_arff_file(input_file, output_file)


Formatted ARFF file saved to: E:\chronic_kidney_disease_full_cleaned.arff


In [6]:
import pandas as pd
from scipy.io import arff

arff_file = r"C:\Users\gnand\Downloads\chronic+kidney+disease\Chronic_Kidney_Disease\Chronic_Kidney_Disease\chronic_kidney_disease_full_cleaned_formatted.arff"
csv_file = r"E:\chronic_kidney_disease_full_cleaned_1.csv"

data, meta = arff.loadarff(arff_file)

# Convert data to pandas DataFrame
df = pd.DataFrame(data)

# Decoding byte strings to regular strings for all object-type columns
for column in df.select_dtypes([object]).columns:
    df[column] = df[column].str.decode('utf-8')

# Save the DataFrame as a CSV file
df.to_csv(csv_file, index=False)

print(f"ARFF file converted to CSV and saved at: {csv_file}")

ARFF file converted to CSV and saved at: E:\chronic_kidney_disease_full_cleaned_1.csv


In [7]:
#START
input_file = r"C:\Users\gnand\Downloads\chronic+kidney+disease\Chronic_Kidney_Disease\Chronic_Kidney_Disease\chronic_kidney_disease_full_cleaned_formatted.arff"
output_file = r"E:\chronic_kidney_disease_full_cleaned_1.csv"

with open(input_file, 'r') as file:
    lines = file.readlines()

formatted_data = []
current_row = []

# Start processing after the @data section
is_data_section = False

for line in lines:
    line = line.strip()  
    if "@data" in line.lower():  # Identify @data section
        is_data_section = True
        formatted_data.append(line)
        continue

    if is_data_section:
        if line:  # Non-empty line
            current_row.append(line)
            if "ckd" in line.lower() or "notckd" in line.lower(): 
                formatted_data.append(",".join(current_row))
                current_row = []


with open(output_file, 'w') as file:
    file.write("\n".join(formatted_data))

print(f"Reformatted file saved to: {output_file}")


Reformatted file saved to: E:\chronic_kidney_disease_full_cleaned_1.csv


In [8]:
import pandas as pd
import re


input_file = r"C:\Users\gnand\Downloads\chronic+kidney+disease\Chronic_Kidney_Disease\Chronic_Kidney_Disease\chronic_kidney_disease_full.arff"
output_file = r"E:\chronic_kidney_disease_full_cleaned_1.csv"

# Open the ARFF file and read 
with open(input_file, 'r') as file:
    lines = file.readlines()

headers = []
data_lines = []
data_start = False

# Loop through the lines to extract headers and data
for line in lines:
    # Extract the headers from @attribute lines
    if line.strip().startswith("@attribute"):
        # header name and store in headers list
        header = re.sub(r"@attribute\s+'(.*?)'.*", r'\1', line.strip())
        headers.append(header)
    
    # after @data section
    if "@data" in line:
        data_start = True
        continue  # Skip the @data line
    
    if data_start:
        data_lines.append(line.strip()) 


data_split = [re.split(r',\s*', line) for line in data_lines]
data_split = [row[:25] for row in data_split]  # first 25 columns of each row

df = pd.DataFrame(data_split, columns=headers)


df.to_csv(output_file, index=False)

print(f"CSV file created successfully at {output_file}!")


CSV file created successfully at E:\chronic_kidney_disease_full_cleaned_1.csv!


In [9]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(r"E:\chronic_kidney_disease_full_cleaned_1.csv")

# Display the first few rows of the data
print(df)

     age   bp     sg   al   su     rbc        pc         pcc          ba  bgr  \
0     48   80  1.020    1    0       ?    normal  notpresent  notpresent  121   
1      7   50  1.020    4    0       ?    normal  notpresent  notpresent    ?   
2     62   80  1.010    2    3  normal    normal  notpresent  notpresent  423   
3     48   70  1.005    4    0  normal  abnormal     present  notpresent  117   
4     51   80  1.010    2    0  normal    normal  notpresent  notpresent  106   
..   ...  ...    ...  ...  ...     ...       ...         ...         ...  ...   
397   12   80  1.020    0    0  normal    normal  notpresent  notpresent  100   
398   17   60  1.025    0    0  normal    normal  notpresent  notpresent  114   
399   58   80  1.025    0    0  normal    normal  notpresent  notpresent  131   
400  NaN  NaN    NaN  NaN  NaN     NaN       NaN         NaN         NaN  NaN   
401  NaN  NaN    NaN  NaN  NaN     NaN       NaN         NaN         NaN  NaN   

     ...  pcv  wbcc rbcc  h

In [4]:

#REPLACING ? WITH NAN


import pandas as pd

df = pd.read_csv(r"E:\chronic_kidney_disease_full_cleaned_1.csv", dtype=str)

df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Remove extra spaces
df.replace('?', pd.NA, inplace=True)

output_path = r"E:\chronic_kidney_disease_full_cleaned_1.csv"
df.to_csv(output_path, index=False)

print(df.head())

  age  bp     sg al su     rbc        pc         pcc          ba  bgr  ...  \
0  48  80  1.020  1  0     NaN    normal  notpresent  notpresent  121  ...   
1   7  50  1.020  4  0     NaN    normal  notpresent  notpresent  NaN  ...   
2  62  80  1.010  2  3  normal    normal  notpresent  notpresent  423  ...   
3  48  70  1.005  4  0  normal  abnormal     present  notpresent  117  ...   
4  51  80  1.010  2  0  normal    normal  notpresent  notpresent  106  ...   

  pcv  wbcc rbcc  htn   dm cad appet   pe  ane class  
0  44  7800  5.2  yes  yes  no  good   no   no   ckd  
1  38  6000  NaN   no   no  no  good   no   no   ckd  
2  31  7500  NaN   no  yes  no  poor   no  yes   ckd  
3  32  6700  3.9  yes   no  no  poor  yes  yes   ckd  
4  35  7300  4.6   no   no  no  good   no   no   ckd  

[5 rows x 25 columns]


C:\Users\gnand\AppData\Local\Temp\ipykernel_9932\3156554375.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Remove extra spaces


In [ ]:
pip install scikit-learn


In [25]:
#without pca
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score


df = pd.read_csv(r"E:\chronic_kidney_disease_full_cleaned_1.csv")

X = df.drop(columns=['class'])
y = df['class']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Identify categorical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=categorical_cols)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Feature selection using RFE with a weaker DecisionTreeClassifier
base_tree = DecisionTreeClassifier(
    random_state=42,
    min_samples_split=10,     
    min_samples_leaf=10,      
    max_depth=4,              
    class_weight="balanced"
)

rfe = RFE(estimator=base_tree, n_features_to_select=4)
X_rfe_numeric = rfe.fit_transform(X_scaled, y_encoded)
X_selected = pd.DataFrame(X_rfe_numeric)

# Define CatBoost classifier with fewer iterations
cat_model = CatBoostClassifier(
    iterations=50,            # Reduced from 100
    learning_rate=0.1,
    depth=5,                 
    verbose=0,
    random_seed=42
)

# Stratified Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
conf_matrices = []
precisions = []
recalls = []
f1_scores = []

# Perform cross-validation and compute metrics
for train_index, test_index in cv.split(X_selected, y_encoded):
    X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    cat_model.fit(X_train, y_train)

    y_train_pred = cat_model.predict(X_train)
    y_test_pred = cat_model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    conf_matrix = confusion_matrix(y_test, y_test_pred, labels=[0, 1])
    precision = precision_score(y_test, y_test_pred, average='weighted')
    recall = recall_score(y_test, y_test_pred, average='weighted')
    f1 = f1_score(y_test, y_test_pred, average='weighted')

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    conf_matrices.append(conf_matrix)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

avg_train_accuracy = np.mean(train_accuracies)
avg_test_accuracy = np.mean(test_accuracies)
avg_conf_matrix = np.sum([np.array(cm) for cm in conf_matrices], axis=0)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1_scores)


print("Testing accuracy:", round(avg_test_accuracy, 3))
print("Training accuracy:", round(avg_train_accuracy, 3))
print("Confusion matrix:")
print(avg_conf_matrix)
print("Precision:", round(avg_precision, 2))
print("Recall:", round(avg_recall, 2))
print("F1-score:", round(avg_f1, 2))


C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing accuracy: 0.973
Training accuracy: 0.986
Confusion matrix:
[[247   0]
 [  0   0]]
Precision: 0.97
Recall: 0.97
F1-score: 0.97


C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [32]:
#with pca
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer, KNNImputer

# Load dataset
df = pd.read_csv(r"E:\chronic_kidney_disease_full_cleaned_1.csv")


X = df.drop(columns=['class'])
y = df['class']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Identify categorical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# Impute missing values (for both categorical and numerical)
imputer = SimpleImputer(strategy='most_frequent')  # For categorical features
X[categorical_cols] = imputer.fit_transform(X[categorical_cols])

# Impute missing numerical features using KNN
knn_imputer = KNNImputer(n_neighbors=5)
X[X.select_dtypes(include=[np.number]).columns] = knn_imputer.fit_transform(X.select_dtypes(include=[np.number]))

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=categorical_cols)

# Separate numerical features from the encoded dataset (after one-hot encoding)
numerical_cols = X_encoded.select_dtypes(include=[np.number]).columns.tolist()

# Scale numerical features
scaler = StandardScaler()
X_scaled = X_encoded[numerical_cols]
X_scaled = scaler.fit_transform(X_scaled)

# Define CatBoost classifier with more iterations and tuned parameters
cat_model = CatBoostClassifier(
    iterations=100,            
    learning_rate=0.05,       
    depth=6,                   
    l2_leaf_reg=5,             
    verbose=0,
    random_seed=42
)

# Stratified Cross-Validation with more splits
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
conf_matrices = []
precisions = []
recalls = []
f1_scores = []

for train_index, test_index in cv.split(X_scaled, y_encoded):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    cat_model.fit(X_train, y_train)

    y_train_pred = cat_model.predict(X_train)
    y_test_pred = cat_model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    conf_matrix = confusion_matrix(y_test, y_test_pred, labels=[0, 1])
    precision = precision_score(y_test, y_test_pred, average='weighted')
    recall = recall_score(y_test, y_test_pred, average='weighted')
    f1 = f1_score(y_test, y_test_pred, average='weighted')

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    conf_matrices.append(conf_matrix)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Compute average values across folds
avg_train_accuracy = np.mean(train_accuracies)
avg_test_accuracy = np.mean(test_accuracies)
avg_conf_matrix = np.sum([np.array(cm) for cm in conf_matrices], axis=0)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1_scores)

print("Testing accuracy:", round(avg_test_accuracy, 3))
print("Training accuracy:", round(avg_train_accuracy, 3))
print("Confusion matrix:")
print(avg_conf_matrix)
print("Precision:", round(avg_precision, 2))
print("Recall:", round(avg_recall, 2))
print("F1-score:", round(avg_f1, 2))


C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing accuracy: 0.968
Training accuracy: 0.977
Confusion matrix:
[[244   0]
 [  0   0]]
Precision: 0.96
Recall: 0.97
F1-score: 0.96


C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
#with pca
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.impute import KNNImputer, SimpleImputer

# Load dataset
df = pd.read_csv(r"E:\chronic_kidney_disease_full_cleaned_1.csv")

# Separate features and labels
X = df.drop(columns=['class'])
y = df['class']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Identify categorical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# Impute missing values (for both categorical and numerical)
imputer = SimpleImputer(strategy='most_frequent')  # For categorical features
X[categorical_cols] = imputer.fit_transform(X[categorical_cols])

# Impute missing numerical features using KNN
knn_imputer = KNNImputer(n_neighbors=5)
X[X.select_dtypes(include=[np.number]).columns] = knn_imputer.fit_transform(X.select_dtypes(include=[np.number]))

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=categorical_cols)

# Separate numerical features from the encoded dataset (after one-hot encoding)
numerical_cols = X_encoded.select_dtypes(include=[np.number]).columns.tolist()

# Scale numerical features
scaler = StandardScaler()
X_scaled = X_encoded[numerical_cols]
X_scaled = scaler.fit_transform(X_scaled)

# Apply PCA only on numerical features
pca = PCA(n_components=4)  # Adjust n_components as needed
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame for the PCA components
pca_columns = [f"PCA_{i+1}" for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(X_pca, columns=pca_columns)

# Drop the original numerical columns from X_encoded and concatenate the PCA results
X_encoded = X_encoded.drop(columns=numerical_cols)
X_encoded = pd.concat([X_encoded, X_pca_df], axis=1)

# Feature selection using RFE with a weaker DecisionTreeClassifier
base_tree = DecisionTreeClassifier(
    random_state=42,
    min_samples_split=10,     # Increased
    min_samples_leaf=10,      # Increased
    max_depth=4,              # Reduced
    class_weight="balanced"
)

# Reduce number of selected features to make it less accurate
rfe = RFE(estimator=base_tree, n_features_to_select=4)
X_rfe_numeric = rfe.fit_transform(X_encoded, y_encoded)
X_selected = pd.DataFrame(X_rfe_numeric)

# Define CatBoost classifier with more iterations and tuned parameters
cat_model = CatBoostClassifier(
    iterations=100,            # Increased from 50
    learning_rate=0.05,        # Reduced from 0.1 for better convergence
    depth=6,                   # Increased from 5
    l2_leaf_reg=5,             # Added L2 regularization
    verbose=0,
    random_seed=42
)

# Stratified Cross-Validation with more splits
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
conf_matrices = []
precisions = []
recalls = []
f1_scores = []

# Perform cross-validation and compute metrics
for train_index, test_index in cv.split(X_selected, y_encoded):
    X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    cat_model.fit(X_train, y_train)

    y_train_pred = cat_model.predict(X_train)
    y_test_pred = cat_model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    conf_matrix = confusion_matrix(y_test, y_test_pred, labels=[0, 1])
    precision = precision_score(y_test, y_test_pred, average='weighted')
    recall = recall_score(y_test, y_test_pred, average='weighted')
    f1 = f1_score(y_test, y_test_pred, average='weighted')

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    conf_matrices.append(conf_matrix)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Compute average values across folds
avg_train_accuracy = np.mean(train_accuracies)
avg_test_accuracy = np.mean(test_accuracies)
avg_conf_matrix = np.sum([np.array(cm) for cm in conf_matrices], axis=0)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1_scores)


print("Testing accuracy:", round(avg_test_accuracy, 3))
print("Training accuracy:", round(avg_train_accuracy, 3))
print("Confusion matrix:")
print(avg_conf_matrix)    
print("Precision:", round(avg_precision, 2))
print("Recall:", round(avg_recall, 2))
print("F1-score:", round(avg_f1, 2))   

C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing accuracy: 0.96
Training accuracy: 0.972
Confusion matrix:
[[241   0]
 [  1   0]]
Precision: 0.96
Recall: 0.96
F1-score: 0.96


C:\Users\gnand\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
